In [172]:
import pandas as pd

In this notebook we will prepare the data.....

### Import data

In [173]:
games = pd.read_csv('games.csv')
games_stats = pd.read_csv('games_stats.csv')

games.rename({'id':'game_id'},axis=1,inplace=True)

### Give a name

In [174]:
# create target varible - home win = 1 , away win = 0

def game_result(h_score, a_score):
    if h_score > a_score:
        return 1
    else:
        return 0

games['result'] = games.apply(lambda x: game_result(x['H_score'], x['A_score']), axis=1)

In [175]:
#join dates info to games_stats dataframe

games_stats[['date','Home_team_id','game_h_team_result']] = \
games_stats.merge(games, on='game_id', how='left' )[['date','H_team_id','result']]

#add result to game_stats 

def game_true_result(team_id, h_team_id, result):
    if team_id == h_team_id:
        return result
    elif result == 0:
        return 1
    else: return 0

games_stats['game_true_result'] = games_stats.apply(lambda x: game_true_result(x['team_id'],x['Home_team_id'],x['game_h_team_result']),axis=1)
games_stats.drop(['Home_team_id','game_h_team_result'],axis = 1, inplace = True)

In [176]:
#win percentage 

games_stats = games_stats.merge(games[['game_id','season']])
games_stats.sort_values(by = ['season','team_id','date'], ascending = True, inplace = True)
team_stats_groupby = games_stats.groupby(['team_id'])

games_stats['game_number'] = team_stats_groupby.cumcount()
games_stats['wins_before'] = team_stats_groupby['game_true_result'].apply(lambda x : x.shift().cumsum())
games_stats['w_per_before'] = games_stats['wins_before']/games_stats['game_number']

games_stats['w_per_last10games'] = team_stats_groupby['game_true_result'] \
                    .rolling(10, closed='left').sum() \
                    .reset_index(drop=True, level=0)
games_stats['w_per_last5games'] = team_stats_groupby['game_true_result'] \
                    .rolling(5, closed='left').sum() \
                    .reset_index(drop=True, level=0) 




In [177]:
# team stats rolling avg 10 games 


columns_to_aggregate = ['fastBreakPoints', 'pointsInPaint', 'biggestLead',
       'secondChancePoints', 'pointsOffTurnovers', 'longestRun', 'points',
       'fgm', 'fga', 'fgp', 'ftm', 'fta', 'ftp', 'tpm', 'tpa', 'tpp', 'offReb',
       'defReb', 'totReb', 'assists', 'pFouls', 'steals', 'turnovers',
       'blocks', 'plusMinus']

columns_aggregated = ['mean_' + col for col in columns_to_aggregate]

games_stats[columns_aggregated] = \
    team_stats_groupby[columns_to_aggregate].rolling(10,  closed='left').mean().reset_index(drop=True, level=0)

relevant_columns = ['game_id','team_id'] + columns_aggregated +['w_per_before','w_per_last10games','w_per_last5games']




In [178]:
# Home team data merge
full_data = games.merge(right=games_stats[relevant_columns].add_prefix('H_'), left_on=['game_id','H_team_id'],right_on=['H_game_id','H_team_id'])
# Away team data merge
full_data = full_data.merge(right=games_stats[relevant_columns].add_prefix('A_'), left_on=['game_id','A_team_id'],right_on=['A_game_id','A_team_id'])

full_data

,game_id,season,date,arena,H_team,H_team_id,A_team,A_team_id,H_score,A_score,...,A_mean_totReb,A_mean_assists,A_mean_pFouls,A_mean_steals,A_mean_turnovers,A_mean_blocks,A_mean_plusMinus,A_w_per_before,A_w_per_last10games,A_w_per_last5games
0,8787,2021,2021-10-19T23:30:00.000Z,Fiserv Forum,Milwaukee Bucks,21,Brooklyn Nets,4,127,104,...,44.2,24.2,19.1,7.7,15.2,5.3,-0.8,0.500000,6.0,3.0
1,8788,2021,2021-10-20T02:00:00.000Z,STAPLES Center,Los Angeles Lakers,17,Golden State Warriors,11,114,121,...,46.8,27.0,20.3,10.7,15.4,4.3,3.8,0.583333,6.0,5.0
2,8789,2021,2021-10-20T23:00:00.000Z,Spectrum Center,Charlotte Hornets,5,Indiana Pacers,15,123,122,...,40.3,23.6,22.8,9.9,16.4,5.8,-6.1,0.428571,3.0,3.0
3,8790,2021,2021-10-20T23:00:00.000Z,Little Caesars Arena,Detroit Pistons,10,Chicago Bulls,6,88,94,...,51.8,25.7,20.5,9.4,14.0,6.0,13.7,0.727273,8.0,5.0
4,8791,2021,2021-10-20T23:30:00.000Z,Madison Square Garden,New York Knicks,24,Boston Celtics,2,138,134,...,43.4,21.2,20.2,7.0,16.9,6.0,-6.9,0.526316,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1606,10800,2021,2021-10-05T00:00:00.000Z,Target Center,Minnesota Timberwolves,22,New Orleans Pelicans,23,117,114,...,43.4,24.8,19.9,7.2,14.8,5.1,1.6,0.437500,4.0,2.0
1607,10801,2021,2021-10-05T00:00:00.000Z,Paycom Center,Oklahoma City Thunder,25,Charlotte Hornets,5,97,113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,NaN,2.0
1608,10802,2021,2021-10-05T00:30:00.000Z,AT&T Center,San Antonio Spurs,31,Utah Jazz,40,111,85,...,42.1,21.1,19.5,6.4,13.2,2.9,5.2,0.476190,5.0,2.0
1609,10803,2021,2021-10-05T02:00:00.000Z,Moda Center,Portland Trail Blazers,29,Golden State Warriors,11,107,121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.285714,NaN,1.0


In [179]:
full_data.to_csv('model_data.csv',index=False)